In [19]:
from utils import load_deepfake_dataset
from dataloader.Loader import Loader
import os
import pandas as pd

class VoxCelebLoader(Loader):
    def read_data(self, directory):
        base = f"{os.getcwd()}/../data/VoxCeleb"
        data_list = []
        for set in directory:
            split = set['split']

            wav_dir = f"{base}/{set['name']}/{set['split']}/wav"
            aac_dir = f"{base}/{set['name']}/{set['split']}/aac"
            if os.path.exists(wav_dir):
                dir = wav_dir
            elif os.path.exists(aac_dir):
                dir = aac_dir

            for root, dirs, files in os.walk(dir):
                for file in files:
                    if not file.endswith(".wav") and not file.endswith(".m4a"):
                        continue
                    utterance = os.path.splitext(
                        os.path.basename(os.path.join(root, file)))[0]
                    id = os.path.join(root, file).split("/")[-3]
                    speaker_id = int(id[2:])
                    data_list.append({
                        'filename': os.path.normpath(os.path.join(root, file)),
                        'utterance': utterance,
                        'speaker': speaker_id,
                        'method_type': 'bonafide',
                        'method_name': 'bonafide',
                        'vocoder': 'bonafide',
                        'is_genuine': 1
                    })
        self.data_list = pd.DataFrame(data_list, columns=[
            'filename', 'utterance', 'speaker', 'method_type', 'method_name', 'vocoder', 'is_genuine',])

def create_csv(name, split):
    ds = [{"name": name, "split": split}]
    loader = VoxCelebLoader(ds, lambda x: x)
    loader.data_list.to_csv(f"../data/VoxCeleb/{name}/{split}/metadata.csv", index=False)

In [17]:
create_csv("VoxCeleb1", "test")

/home/patrick/speaker_recognition_light/source/../data/VoxCeleb/VoxCeleb1/test/wav
00001.wav
00002.wav
00003.wav
00004.wav
00005.wav
00006.wav
00007.wav
00008.wav
00009.wav
00010.wav
00011.wav
00012.wav
00013.wav
00014.wav
00015.wav
00016.wav
00017.wav
00018.wav
00019.wav
00020.wav
00021.wav
00022.wav
00023.wav
00024.wav
00025.wav
00026.wav
00027.wav
00001.wav
00002.wav
00001.wav
00002.wav
00003.wav
00004.wav
00005.wav
00006.wav
00007.wav
00008.wav
00009.wav
00010.wav
00011.wav
00012.wav
00013.wav
00014.wav
00015.wav
00016.wav
00017.wav
00018.wav
00019.wav
00020.wav
00021.wav
00022.wav
00023.wav
00024.wav
00025.wav
00026.wav
00027.wav
00028.wav
00029.wav
00030.wav
00031.wav
00032.wav
00033.wav
00034.wav
00035.wav
00036.wav
00037.wav
00038.wav
00001.wav
00002.wav
00003.wav
00004.wav
00005.wav
00006.wav
00007.wav
00008.wav
00009.wav
00010.wav
00011.wav
00012.wav
00013.wav
00014.wav
00015.wav
00016.wav
00017.wav
00018.wav
00019.wav
00020.wav
00021.wav
00022.wav
00023.wav
00024.wav
00025.w

In [ ]:
create_csv("VoxCeleb2", "test")

In [3]:
create_csv("VoxCeleb1", "dev")

In [12]:
create_csv("VoxCeleb2", "dev")

KeyboardInterrupt: 